<a href="https://colab.research.google.com/github/DonRomaniello/citibike_dock_availability/blob/master/TarAppend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import time

import json
import tarfile
import os

In [0]:
bikepump = pd.read_pickle('./Google Drive/GeneralAssembly/Citibike/Processed/Old.pkl.xz', compression='xz')
directory = './Google Drive/GeneralAssembly/Citibike/Processed/Processme'

In [0]:
for filename in os.listdir(directory):
  if '.tar.xz' not in filename:
    pass
  else:
    hour_path = directory + '/' + filename
    hour = tarfile.open(hour_path)
    hour_list = hour.getnames()

    for subfile in hour_list:
      filthy_data = json.load(hour.extractfile(subfile))
      timestamp = filthy_data["last_updated"]
      bikepump2 = pd.DataFrame(filthy_data['data']['stations'])

      #Sort by station_id
      bikepump2.sort_values(by=['station_id'], ascending=True, inplace=True)

      #Make a list of columns to remove (all but two)
      pre_drop = list(bikepump2.columns)

      # Set the index to be Station ID
      bikepump2.set_index(['station_id'], drop=True, inplace=True)

      # Remove the columns we want from the list
      pre_drop.remove('station_id')
      pre_drop.remove('num_docks_available')
      bikepump2.drop(pre_drop, inplace=True, axis=1)

      #Name the columns according to the timestamp on the data
      bikepump2.rename(columns={'num_docks_available' : timestamp}, inplace=True)

      if bikepump.empty:
        bikepump = bikepump2

      # Merge the columns, use 'inner' because this means defunct stations are not included
      if timestamp not in bikepump.columns:
        bikepump = bikepump.merge(bikepump2, how='inner', on='station_id')

In [0]:
print(bikepump.info())

<class 'pandas.core.frame.DataFrame'>
Index: 927 entries, 116 to 83
Columns: 52366 entries, 1580684459 to 1582437599
dtypes: int64(52366)
memory usage: 370.4+ MB
None


In [0]:
#Export; despite security concerns I'm using 'pkl' because when I ran json export and import the UNIX time was converted
bikepump.to_csv('./Google Drive/GeneralAssembly/Citibike/Processed/Old.csv')
bikepump.to_pickle('./Google Drive/GeneralAssembly/Citibike/Processed/Old.pkl')

In [0]:
bikepump.loc["Total Availability"] = bikepump.sum()

In [0]:
# Round epoch to nearest 60 (iles were named based on when wget managed to finish downloading...)

round_docks = bikepump.copy(deep=True)
round_docks = round_docks.reindex(sorted(round_docks.columns), axis=1)
epoch_rename_feed = list(round_docks.columns)
epoch_rename_results = []

def myround(x, base=60):
    return int(base * round(float(x)/base))

for sloppy in range(len(epoch_rename_feed)):
  if (epoch_rename_feed[sloppy] % 60) != 0:
    epoch_rename_feed[sloppy]=myround(epoch_rename_feed[sloppy])

round_docks.columns = epoch_rename_feed
round_docks.to_csv('./Google Drive/GeneralAssembly/Citibike/Processed/OldRound.csv')

In [0]:
#Transpose

flip_docks = pd.DataFrame(columns=['TA', 'DT'])
flip_docks['TA']=round_docks.loc['Total Availability']
flip_docks['DT']=pd.to_datetime(round_docks.columns, unit='s', utc=False)
t_docks = round_docks.T
t_docks.to_csv('./Google Drive/GeneralAssembly/Citibike/Processed/OldRoundTransposed.csv')

In [0]:
!xz -zf --threads=0 ./Google\ Drive/GeneralAssembly/Citibike/Processed/Old.csv
!xz -zf --threads=0 ./Google\ Drive/GeneralAssembly/Citibike/Processed/Old.pkl
!xz -zf --threads=0 ./Google\ Drive/GeneralAssembly/Citibike/Processed/OldRound.csv
!xz -zf --threads=0 ./Google\ Drive/GeneralAssembly/Citibike/Processed/OldRoundTransposed.csv